Manipulators are verbs

In [ ]:
class Pixel(x: Int, y: Int) {
  def paint(color: Color): Unit = {
    // Implementation here
  }
}

val center = new Pixel(50, 50)
center.paint(new Color("red"))

When an object allows us to manipulate, the name has to be a verb, and there must be no return value.

Examples of refactoring


In [ ]:
class Document {
  def write(content: InputStream): Unit = {
    // Implementation here
  }
}

The example above shows a Document class with a write method that serves as a manipulator. Following the principle, it has a verb name and returns Unit (void).

However, what if we need to know how many bytes were saved? Let's refactor according to the principles:


In [ ]:
// Problem: This violates our principle - a manipulator returning a value
class DocumentProblem {
  def write(content: java.io.InputStream): Int = {
    // Implementation that writes and returns bytes written
    42 // Dummy return for example
  }
}

// Solution: Separate the concerns
class Document {
  def output(): OutputPipe = {
    new OutputPipe()
  }
}

class OutputPipe {
  def write(content: java.io.InputStream): Unit = {
    // Implementation here
  }

  def bytes: Int = {
    // Return number of bytes written
    42 // Dummy return for example
  }

  def time: Long = {
    // Return time taken for writing
    123L // Dummy return for example
  }
}

// Usage example
val doc = new Document()
val pipe = doc.output()
pipe.write(new java.io.ByteArrayInputStream("Hello".getBytes))
val bytesWritten = pipe.bytes
val timeElapsed = pipe.time

Notice how we've refactored to maintain the principle:
1. `output()` is a builder - it returns a new object and has a noun name
2. `write()` is a manipulator - it has a verb name and returns Unit
3. `bytes` and `time` are builders - they have noun names and return values


The Builder Pattern and its concerns


In [ ]:
// Builder Pattern example (though the author recommends against it)
class Book private (
  val author: String = "",
  val title: String = "",
  val pages: List[String] = List.empty
) {
  def withAuthor(author: String): Book = {
    new Book(author, title, pages)
  }

  def withTitle(title: String): Book = {
    new Book(author, title, pages)
  }

  def withPage(page: String): Book = {
    new Book(author, title, pages :+ page)
  }
}

// Usage
val myBook = new Book()
  .withAuthor("John Doe")
  .withTitle("Clean Code")
  .withPage("Chapter 1")

The Builder Pattern methods follow the principle - they have noun-like names (with a prefix) and return values. However, as noted in the text, the author generally advises against this pattern as it often leads to less cohesive, larger objects.


A better approach to complex objects with many properties


In [ ]:
// Instead of a large Book class with a builder, break it into smaller objects
class Author(val name: String)
class Title(val text: String)
class Page(val content: String)

class Book(val author: Author, val title: Title, val pages: List[Page]) {
  def printInfo(): Unit = {
    println(s"${title.text} by ${author.name}, ${pages.size} pages")
  }
}

// Usage
val book = new Book(
  new Author("Jane Smith"),
  new Title("Elegant Objects"),
  List(new Page("Introduction"), new Page("Chapter 1"))
)
book.printInfo()

This approach creates smaller, more focused objects rather than using the Builder Pattern for complex initialization.


Avoiding Multiple Return Values


In [ ]:
// In some languages like Go, multiple returns might be used:
// func write(content) (int bytes, int time) { ... }

// In Scala, we isolate the concept in its own class
class Transaction {
  def execute(content: java.io.InputStream): Unit = {
    // Execute the write operation
  }

  def statistics: Statistics = {
    new Statistics(42, 123L) // bytes and time
  }
}

class Statistics(val bytesWritten: Int, val executionTimeMs: Long)

// Usage
val transaction = new Transaction()
transaction.execute(new java.io.ByteArrayInputStream("Data".getBytes))
val stats = transaction.statistics
println(s"Wrote ${stats.bytesWritten} bytes in ${stats.executionTimeMs}ms")

By following these principles consistently, we create more focused objects with clearer responsibilities, making the code more maintainable and easier to understand.


Summary:
1. Manipulators have verb names and return nothing (Unit in Scala)
2. Builders have noun names and return values
3. Keep methods focused on a single responsibility
4. Prefer smaller, more cohesive objects over large complex ones
